In [1]:
# universally modules
import sys
sys.path.append("../src")
import numpy as np
import pandas as pd

# preprocessing and transformation modules
import fasttext
import Preprocessing
from Features import buildFeatures
from Pipeline import BuildPipeline
from Modelling import StackingModelling
from ModelSelection import ModelSelection, process_case
from Transformation import StackedTransformation, transformation

# model algorithm
from sklearn.svm import LinearSVC, LinearSVR
from xgboost import XGBRegressor, XGBClassifier
from sklearn.linear_model import SGDClassifier, SGDRegressor


/home/bigdata/anaconda3/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
# read datasets:
df_full_preprocessed = pd.read_pickle("./df_full_preprocessed.pkl")
df_eval_results_age =  pd.read_pickle("./Model Selection/pd_df_cv_age_2020-12-27 09:47:05.199850.pkl")
df_eval_results_sign =  pd.read_pickle("./Model Selection/pd_df_cv_sign_2020-12-27 07:12:32.465884.pkl")
df_eval_results_topic =  pd.read_pickle("./Model Selection/pd_df_cv_topic_2021-01-04 19:33:32.751170.pkl")
df_eval_results_gender =  pd.read_pickle("./Model Selection/pd_df_cv_gender_2020-12-25 00:31:53.679084.pkl")

## Pipeline erstellen

In [ ]:
age_model = BuildPipeline(df_eval_results_age, df_full_preprocessed, "regression")
age_model.fit()

gender_model = BuildPipeline(df_eval_results_gender, df_full_preprocessed, "classification")
gender_model.fit()

sign_model = BuildPipeline(df_eval_results_sign, df_full_preprocessed, "classification")
sign_model.fit()

topic_model = BuildPipeline(df_eval_results_topic, df_full_preprocessed, "classification")
topic_model.fit()

In [ ]:
# save models to series
model_pipelines = pd.Series([age_model, gender_model, sign_model, topic_model])

In [ ]:
model_pipelines.to_pickle("./Pipelines/ModelPipelines.pkl")